In [1]:
import sys
sys.path.insert(0, 'C:/Users/14432/OneDrive/Research/Projects/A549_HMGA1/src/4_wk_trajectory/')
sys.path.insert(1, 'C:/Users/14432/OneDrive/Research/Projects/A549_HMGA1/src/3-compute/tools/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pickle
from PIL import Image, ImageDraw, ImageFont

from matplotlib import animation
from matplotlib import cm
from matplotlib import pyplot as plt

import os
from os import listdir

import scipy.io as sio
import scipy.misc
import scipy.ndimage as ndimage
from scipy.signal import medfilt,find_peaks
from scipy.stats import kde
import scipy.interpolate.fitpack as fitpack

from skimage.feature import peak_local_max
from skimage import measure
from skimage.measure import regionprops, label
from skimage.morphology import opening,closing
from skimage.io import imread
from skimage.segmentation import find_boundaries

from sklearn import manifold,decomposition,random_projection,cluster,metrics,preprocessing,mixture,model_selection
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler

from cell_class import single_cell,fluor_single_cell
import image_warp
from contour_tool import df_find_contour_points,find_contour_points,generate_contours,align_contour_to,align_contours
import pipe_util2
import utility_tools
import contour_class
from traj_class import single_cell_traj,fluor_single_cell_traj

In [2]:
def generate_single_cell_img(img,seg,img_num,obj_num):      
    #single_obj_mask=morphology.binary_dilation(seg==obj_num,morphology.disk(6))
    single_obj_mask=seg==obj_num
    single_obj_mask=label(single_obj_mask)
    rps=regionprops(single_obj_mask)
    candi_r=[r for r in rps if r.label==1][0]
    candi_box=candi_r.bbox        
    single_cell_img=single_obj_mask*img
    
    crop_cell_img=single_cell_img[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]
    crop_cell_img_env=img[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]
    
    crop_single_obj_mask=single_obj_mask[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]
    rps=regionprops(crop_single_obj_mask)
    candi_r=[r for r in rps if r.label==1][0]
    center=candi_r.centroid
    
    return crop_cell_img,crop_cell_img_env

In [3]:
def find_mother(df,mitosis_df,traj_label):
    daughter_flag=0
    if traj_label!=-1:
        if (mitosis_df['sis1_traj_label']==traj_label).any() or (mitosis_df['sis2_traj_label']==traj_label).any():
            daughter_flag=1
    return daughter_flag

In [4]:
def find_offspring(df,mitosis_df,family_tree,traj_label):
    mother_label=traj_label
    if mother_label!=-1 and (mitosis_df['mother_traj_label']==mother_label).any():
        family_tree[int(mother_label)]=[]
        sis1_label=mitosis_df.loc[mitosis_df['mother_traj_label']==mother_label,'sis1_traj_label'].values[0]
        sis2_label=mitosis_df.loc[mitosis_df['mother_traj_label']==mother_label,'sis2_traj_label'].values[0]
        if sis1_label!=-1:
            family_tree[int(mother_label)].append(int(sis1_label))
        if sis2_label!=-1:
            family_tree[int(mother_label)].append(int(sis2_label))
        family_tree=find_offspring(df,mitosis_df,family_tree,sis1_label)
        family_tree=find_offspring(df,mitosis_df,family_tree,sis2_label)
        return family_tree
    else:
        return family_tree

In [5]:
def parse(node, tree):
    if node not in tree:
        yield [node]
    else:
        for next_node in tree[node]:
            for r in parse(next_node,tree):
                yield [node]+r

In [6]:
def find_abnormal_fluor(traj_fluor,traj_t,peak_h=5):
    mask=traj_fluor!=0
#     inds=np.where(traj_fluor!=0)[0]
    non0_traj_t=traj_t[mask]
    non0_traj_fluor=traj_fluor[mask]
    mean_fluct=np.mean(abs(np.diff(non0_traj_fluor)))
    
    ind1=find_peaks(np.diff(non0_traj_fluor)/mean_fluct,height=peak_h)[0]+1
    ind2=non0_traj_fluor.shape[0]-(find_peaks(np.diff(np.flip(non0_traj_fluor,0))/mean_fluct,height=peak_h)[0])-2
    inds=np.unique(np.concatenate((ind1,ind2)))

    abn_t=non0_traj_t[inds]
    abn_inds=np.where(np.in1d(traj_t,abn_t))[0]#find index of abn_t in traj_t
    return abn_inds

In [7]:
def count_num_fluor_pca_cord(cells,fluor_name,fluor_feature_name):
    for single_cell in cells:
        if hasattr(single_cell,fluor_name+'_feature_values'):
            print('this cell has pca cord')
            num_fluor_pca_cord = len(single_cell.vimentin_haralick_pca_cord)
            print(num_fluor_pca_cord)
            break
    return num_fluor_pca_cord

In [8]:
def generate_fluor_long_traj(df,cells,am_record,traj_df,traj,fluor_name,feature_list,fluor_feature_name):
    
    haralick_labels = ["Angular Second Moment",
                   "Contrast",
                   "Correlation",
                   "Sum of Squares: Variance",
                   "Inverse Difference Moment",
                   "Sum Average",
                   "Sum Variance",
                   "Sum Entropy",
                   "Entropy",
                   "Difference Variance",
                   "Difference Entropy",
                   "Information Measure of Correlation 1",
                   "Information Measure of Correlation 2",
                    "Maximal Correlation Coefficient"]
    
    
    traj_record=pd.DataFrame(traj_df.loc[:,'1':str(time_span)])
    traj_record=traj_record.values
    traj_quan,traj_len=traj_record.shape[0],traj_record.shape[1]
    
    traj_xy=[]
    traj_feature=[]
    traj_contour=[]
    traj_cord=[]
    traj_seri=[]
    traj_am_flag=[]
 
    traj_fluor_feature_values=[]
    traj_haralick=[]
    traj_norm_haralick=[]
    traj_fluor_pca_cord=[]

    num_fluor_pca_cord=count_num_fluor_pca_cord(cells,fluor_name,fluor_feature_name)
    for img_num in range(1,traj_len+1):
        obj_num=traj[img_num-1]
        if obj_num!=-1:
            ind=df.loc[(df['ImageNumber']==img_num)&(df['ObjectNumber']==obj_num)].index.tolist()[0]

            if hasattr(cells[ind],'cell_contour') and hasattr(cells[ind],'pca_cord'):
                traj_contour.append(cells[ind].cell_contour.points.flatten())
                traj_cord.append(cells[ind].pca_cord)
                traj_seri.append([img_num,obj_num])
                traj_xy.append([df.loc[ind,'Cell_AreaShape_Center_X'],df.loc[ind,'Cell_AreaShape_Center_Y']])
                traj_feature.append(df.loc[ind,'Cell_AreaShape_Area':'Cell_AreaShape_Solidity'].values.tolist())
                
                if ((am_record['ImageNumber']==img_num)&(am_record['ObjectNumber']==obj_num)).any():
                    am_flag=np.asscalar(am_record.loc[(am_record['ImageNumber']==img_num)&(am_record['ObjectNumber']==obj_num),'am_flag'].values)
                    traj_am_flag.append(am_flag)
                else:
                    traj_am_flag.append(0)

                if hasattr(cells[ind],fluor_name+'_feature_values'):
                    exec('traj_fluor_feature_values.append(np.array(cells[ind].'+fluor_name+'_feature_values[:3]))')
                    exec('traj_haralick.append(np.array(cells[ind].'+fluor_name+'_feature_values[3]))')
                    exec('traj_norm_haralick.append(np.array(cells[ind].'+fluor_name+'_feature_values[4]))')
                    exec('traj_fluor_pca_cord.append(cells[ind].'+fluor_feature_name[0]+'_pca_cord)')
                else:                    
                    traj_fluor_feature_values.append(np.zeros((3,)))
                    traj_haralick.append((np.zeros(13,)))
                    traj_norm_haralick.append((np.zeros(13,)))
                    traj_fluor_pca_cord.append(np.zeros((num_fluor_pca_cord,)))
               
    
    traj_xy=np.asarray(traj_xy)
    traj_feature=np.asarray(traj_feature)
    traj_contour=np.asarray(traj_contour)
    traj_cord=np.asarray(traj_cord)
    traj_seri=np.asarray(traj_seri)
    
    traj_am_flag=np.asarray(traj_am_flag)
    
    traj_fluor_feature_values=np.asarray(traj_fluor_feature_values)
    
    traj_haralick=np.asarray(traj_haralick)
    traj_norm_haralick=np.asarray(traj_norm_haralick)
    traj_fluor_pca_cord=np.asarray(traj_fluor_pca_cord)    
   
    mask=traj_fluor_feature_values[:,0]!=0
        
    abn_inds=find_abnormal_fluor(traj_fluor_feature_values[:,0],traj_seri[:,0])
    if len(abn_inds)>0:
        traj_fluor_feature_values[abn_inds,:]=0
        traj_haralick[abn_inds,:]=0
        traj_norm_haralick[abn_inds,:]=0
        print(traj_fluor_pca_cord.shape)
        traj_fluor_pca_cord[abn_inds,:]=0

    return traj_feature,traj_contour,traj_cord,traj_seri,traj_am_flag,\
           [traj_fluor_feature_values[:,0],traj_fluor_feature_values[:,1],traj_fluor_feature_values[:,2],\
           traj_haralick,traj_norm_haralick],traj_fluor_pca_cord

In [9]:
main_path='C:/Users/14432/OneDrive/Research/Projects/A549_144hr/data/'
output_path=main_path+'out/11-2-21_mt_20E-3_lt_35E-2/'
sct_path=output_path+'sct/'
if not os.path.exists(sct_path):
    os.makedirs(sct_path)

# 5 fr buffer
# posi_end=21
# time_span=225
# traj_len_thres=67

# begin_range=151
# end_range=219

# 0 fr buffer
posi_end=3
buffer_frames = 0
time_span=1729
traj_len_thres=864
frag_len_thres=traj_len_thres-buffer_frames

begin_range=time_span-traj_len_thres 
end_range=time_span-buffer_frames 

loss_ratio_thres=0.5

In [10]:
feature_list=['mean_intensity','std_intensity','intensity_range','haralick','norm_haralick']
fluor_feature_name=['vimentin_haralick','norm_vimentin_haralick']
for posi in range(1,posi_end+1):
    posi_label = 'XY' + str(posi)
    print(posi_label)
    
    dir_path=output_path+posi_label+'/'
     
    df=pd.read_csv(dir_path+'Per_Object_relink.csv')
    am_record=pd.read_csv(dir_path+'am_record.csv')
    traj_df=pd.read_csv(dir_path+'traj_object_num.csv')
    mitosis_df=pd.read_csv(dir_path+'mitosis_record.csv')
    with open (dir_path+'cells/'+'fluor_cells', 'rb') as fp:
        cells = pickle.load(fp)

    traj_record=pd.DataFrame(traj_df.loc[:,'1':str(time_span)])
    traj_record=traj_record.values
    traj_quan,traj_len=traj_record.shape[0],traj_record.shape[1]

    for traj_label in range(1,traj_quan+1):
        print(traj_label)
        cur_traj=traj_record[traj_label-1,:]
        traj_start_t=np.where(cur_traj!=-1)[0][0]+1
        daughter_flag=find_mother(df,mitosis_df,traj_label)
        if daughter_flag==1:
            continue
        if traj_start_t<=begin_range:
            family_tree={}
            family_tree=find_offspring(df,mitosis_df,family_tree,traj_label=traj_label)
            print('family',family_tree)
            all_branches=list(list(parse(traj_label, family_tree)))
            print('all branch',all_branches)

            for branch in all_branches:   
                branch_end_label=branch[-1]
                branch_end_traj=traj_record[branch_end_label-1,:]
                branch_end_t=np.where(branch_end_traj!=-1)[0][-1]+1
                if branch_end_t>=end_range:
                    long_traj=-1*np.ones((time_span,))
                    divide_points=[]
                    traj_name=''
                    for sub_label in branch:
                        sub_traj=traj_record[sub_label-1,:]
                        mask=sub_traj!=-1
                        if (np.where(sub_traj!=-1)[0][-1]+1)!=branch_end_t:
                            divide_points.append(np.where(sub_traj!=-1)[0][-1]+1)
                        long_traj[mask]=sub_traj[mask]
                        traj_name=traj_name+str(sub_label)+'_'

                    whole_branch=long_traj[traj_start_t-1:branch_end_t]
                    if whole_branch.shape[0] == 0:
                        print(traj_start_t, branch_end_t)

                    long_traj_loss_ratio=whole_branch[whole_branch==-1].shape[0]*1.0/whole_branch.shape[0]
                    print(whole_branch.shape[0])
                    if long_traj_loss_ratio<loss_ratio_thres:
                    
                        traj_feature,traj_contour,\
                            traj_cord,traj_seri,traj_am_flag,\
                            traj_fluor_feature_values,traj_fluor_pca_cord=generate_fluor_long_traj(
                            df,cells,am_record,\
                            traj_df,long_traj,'vimentin',\
                            feature_list,fluor_feature_name)
                        traj_sct=fluor_single_cell_traj(traj_seri,traj_contour)
                        traj_sct.set_traj_feature(traj_feature)
                        traj_sct.set_traj_cord(traj_cord)
                        traj_sct.set_traj_divide_points(np.array(divide_points))
                        traj_sct.set_traj_am_flag(traj_am_flag)
                        traj_sct.set_traj_fluor_features('vimentin',feature_list,traj_fluor_feature_values)
                        traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[0],traj_fluor_pca_cord)
                        
                        if branch_end_t == time_span:
                            if branch_end_label==traj_label:
                                with open(sct_path+'fluor_sct_'+posi_label+'_'+traj_name+'long_traj', 'wb') as fp:
                                    pickle.dump(traj_sct, fp)
                            else:
                                with open(sct_path+'indirect_fluor_sct_'+posi_label+'_'+traj_name+'long_traj', 'wb') as fp:
                                    pickle.dump(traj_sct, fp)
                                    
                        else:
                            if branch_end_label==traj_label:
                                with open(sct_path+'frag_fluor_sct_'+posi_label+'_'+traj_name+'long_traj_label_'+
                                          str(branch_end_label)+'_et_'+str(branch_end_t)+
                                          '_obj_'+str(int(traj_seri[-1][1])), 'wb') as fp:
                                    pickle.dump(traj_sct, fp)
                            else:
                                with open(sct_path+'frag_indirect_fluor_sct_'+posi_label+'_'+traj_name+'long_traj'+
                                          str(branch_end_label)+'_et_'+str(branch_end_t)+
                                          '_obj_'+str(int(traj_seri[-1][1])), 'wb') as fp:
                                    pickle.dump(traj_sct, fp)

#                         fig=plt.figure()
#                         ims=[]
#                         for img_num,obj_num in traj_seri[traj_fluor_feature_values[0]>0]:
#                             img_num=img_num.astype(np.int)
#                             obj_num=obj_num.astype(np.int)
#                             if obj_num!=-1:
#                                 img=imread(img_path+img_list[img_num-1])
#                                 seg_img=imread(seg_path+seg_list[img_num-1])
#                                 crop_cell,crop_cell_env=generate_single_cell_img(img,seg_img,img_num,obj_num)
#                                 im=plt.imshow(crop_cell,animated=True)
#                                 ims.append([im])
#                         ani = animation.ArtistAnimation(fig, ims, interval=150)
#                         if branch_end_label==traj_label:
#                             ani.save(sct_path+'morph_sct_xy'+str(posi)+'_'+traj_name+'long_traj.mp4')
#                         else:
#                             ani.save(sct_path+'indirect_morph_sct_xy'+str(posi)+'_'+traj_name+'long_traj.mp4')
                        
#                         board_size=250
    
#                         fig=plt.figure()
#                         ims=[]
#                         for img_num,obj_num in traj_seri[traj_fluor_feature_values[0]>0]:
#                             img_num=img_num.astype(np.int)
#                             obj_num=obj_num.astype(np.int)
#                             if obj_num!=-1:
#                                 img=imread(fluor_img_path+fluor_img_list[img_num-1])
#                                 seg_img=imread(seg_path+seg_list[img_num-1])
#                                 crop_cell,crop_cell_env=generate_single_cell_img(img,seg_img,img_num,obj_num)
#                                 x_c,y_c=crop_cell.shape[0]//2,crop_cell.shape[1]//2
#                                 x_l,y_l=board_size/2-x_c,board_size/2-y_c
#                                 cell_on_board=np.zeros((board_size,board_size))
#                                 cell_on_board[x_l:x_l+crop_cell.shape[0],y_l:y_l+crop_cell.shape[1]]=crop_cell
                                
#                                 im=plt.imshow(cell_on_board,animated=True)
#                                 plt.axis('off')
#                                 ims.append([im])
                                
#                         ani = animation.ArtistAnimation(fig, ims, interval=100)
#                         if branch_end_label==traj_label:
#                             ani.save(sct_path+'fluor_sct_xy'+str(posi)+'_'+traj_name+'long_traj.mp4')
#                         else:
#                             ani.save(sct_path+'indirect_fluor_sct_xy'+str(posi)+'_'+traj_name+'long_traj.mp4')

XY01
1
family {}
all branch [[1]]
2
family {2: [34, 35]}
all branch [[2, 34], [2, 35]]
361
this cell has pca cord
6
(358, 6)
3
family {}
all branch [[3]]
4
family {4: [30, 31]}
all branch [[4, 30], [4, 31]]
5
family {}
all branch [[5]]
6
family {6: [43, 44]}
all branch [[6, 43], [6, 44]]
361
this cell has pca cord
6
(361, 6)
7
family {}
all branch [[7]]
8
family {}
all branch [[8]]
9
family {}
all branch [[9]]
10
family {}
all branch [[10]]
11
family {}
all branch [[11]]
12
family {}
all branch [[12]]
13
family {}
all branch [[13]]
14
family {14: [21, 22], 21: [37, 38], 22: [53, 51], 51: [66, 65]}
all branch [[14, 21, 37], [14, 21, 38], [14, 22, 53], [14, 22, 51, 66], [14, 22, 51, 65]]
15
family {}
all branch [[15]]
16
family {}
all branch [[16]]
17
family {}
all branch [[17]]
18
family {18: [48, 49]}
all branch [[18, 48], [18, 49]]
19
family {}
all branch [[19]]
20
family {}
all branch [[20]]
21
22
23
family {}
all branch [[23]]
24
family {}
all branch [[24]]
25
family {25: [27, 26]}


82
family {}
all branch [[82]]
83
family {}
all branch [[83]]
84
family {}
all branch [[84]]
85
family {}
all branch [[85]]
86
family {}
all branch [[86]]
87
family {}
all branch [[87]]
88
family {}
all branch [[88]]
89
family {}
all branch [[89]]
90
family {}
all branch [[90]]
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
XY04
1
family {}
all branch [[1]]
361
this cell has pca cord
6
(360, 6)
2
family {2: [44, 45]}
all branch [[2, 44], [2, 45]]
3
family {}
all branch [[3]]
4
family {}
all branch [[4]]
5
family {}
all branch [[5]]
6
family {}
all branch [[6]]
7
family {}
all branch [[7]]
8
family {8: [83, 84]}
all branch [[8, 83], [8, 84]]
361
this cell has pca cord
6
(355, 6)
9
family {}
all branch [[9]]
10
family {10: [67, 66], 67: [195, 198]}
all branch [[10, 67, 195], [10, 67, 198], [10, 66]]
361
this cell has pca cord
6
(360, 6)
361
this cell has pca cord
6
(355, 6)
11
family {}
all branch [[11]]
12
famil

82
family {}
all branch [[82]]
83
family {}
all branch [[83]]
84
family {84: [100, 99]}
all branch [[84, 100], [84, 99]]
142
this cell has pca cord
6
85
family {}
all branch [[85]]
86
family {}
all branch [[86]]
128
this cell has pca cord
6
(124, 6)
87
family {87: [95]}
all branch [[87, 95]]
88
family {}
all branch [[88]]
89
family {}
all branch [[89]]
90
family {}
all branch [[90]]
91
family {}
all branch [[91]]
92
family {}
all branch [[92]]
91
this cell has pca cord
6
93
family {}
all branch [[93]]
95
this cell has pca cord
6
(92, 6)
94
family {}
all branch [[94]]
95
96
family {}
all branch [[96]]
97
98
99
100
101
family {}
all branch [[101]]
102
family {}
all branch [[102]]
103
family {}
all branch [[103]]
104
105
106
family {}
all branch [[106]]
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166

216
family {}
all branch [[216]]
81
this cell has pca cord
6
217
family {}
all branch [[217]]
218
family {}
all branch [[218]]
219
220
family {220: [231, 233]}
all branch [[220, 231], [220, 233]]
221
222
family {}
all branch [[222]]
77
this cell has pca cord
6
(68, 6)
223
family {}
all branch [[223]]
224
family {}
all branch [[224]]
225
family {}
all branch [[225]]
226
family {}
all branch [[226]]
73
this cell has pca cord
6
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
XY07
1
family {}
all branch [[1]]
2
family {2: [36, 37]}
all branch [[2, 36

(291, 6)
38
family {}
all branch [[38]]
39
family {}
all branch [[39]]
40
family {}
all branch [[40]]
41
family {}
all branch [[41]]
42
family {}
all branch [[42]]
43
family {}
all branch [[43]]
44
family {}
all branch [[44]]
45
family {}
all branch [[45]]
46
family {}
all branch [[46]]
47
family {}
all branch [[47]]
48
family {}
all branch [[48]]
49
family {}
all branch [[49]]
50
family {}
all branch [[50]]
51
family {}
all branch [[51]]
52
family {52: [87, 86]}
all branch [[52, 87], [52, 86]]
53
family {53: [288, 289]}
all branch [[53, 288], [53, 289]]
260
this cell has pca cord
6
(257, 6)
260
this cell has pca cord
6
(257, 6)
54
family {}
all branch [[54]]
55
family {}
all branch [[55]]
56
family {}
all branch [[56]]
57
family {}
all branch [[57]]
58
family {}
all branch [[58]]
59
family {}
all branch [[59]]
60
family {60: [100, 101]}
all branch [[60, 100], [60, 101]]
256
this cell has pca cord
6
(246, 6)
61
family {}
all branch [[61]]
62
family {62: [91, 90], 90: [144, 143]}
all br

...

418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
XY14
1
family {}
all branch [[1]]
361
this cell has pca cord
6
(359, 6)
2
family {2: [92, 93]}
all branch [[2, 92], [2, 93]]
361
this cell has pca cord
6
(359, 6)
361
this cell has pca cord
6
(360, 6)
3
family {3: [38, 39], 38: [179, 178]}
all branch [[3, 38, 179], [3, 38, 178], [3, 39]]
361
this cell has pca cord
6
(347, 6)
361
this cell has pca cord
6
(347, 6)
4
family {4: [72, 71]}
all branch [[4, 72], [4, 71]]
5
family {}
all branch [[5]]
6
family {6: [44, 43]}
all branch [[6, 44], [6, 43]]
361
this cell has pca cord
6
(353, 6)
7
family {}
all branch [[7]]
8
family {}
all branch [[8]]
9
family {}
all branch [[9]]
361
this cell has pca cord
6
10
family {}
all branch [[10]]
361
this cell has pca cord
6
11
family {}
all branch [[11]]
12
family {}
all branch [[12]]
13
family {}
all branch [[13

95
family {}
all branch [[95]]
96
family {}
all branch [[96]]
187
this cell has pca cord
6
(184, 6)
97
family {}
all branch [[97]]
98
family {}
all branch [[98]]
99
family {}
all branch [[99]]
100
family {}
all branch [[100]]
101
family {101: [161, 160]}
all branch [[101, 161], [101, 160]]
102
family {}
all branch [[102]]
103
family {}
all branch [[103]]
104
family {}
all branch [[104]]
105
family {}
all branch [[105]]
106
family {}
all branch [[106]]
107
family {}
all branch [[107]]
172
this cell has pca cord
6
108
family {}
all branch [[108]]
109
family {}
all branch [[109]]
110
family {}
all branch [[110]]
111
family {}
all branch [[111]]
112
family {}
all branch [[112]]
113
family {}
all branch [[113]]
114
family {}
all branch [[114]]
115
family {115: [184, 185], 185: [203, 201]}
all branch [[115, 184], [115, 185, 203], [115, 185, 201]]
116
family {}
all branch [[116]]
162
this cell has pca cord
6
117
family {}
all branch [[117]]
118
family {}
all branch [[118]]
159
this cell has p

167
this cell has pca cord
6
(165, 6)
142
family {}
all branch [[142]]
143
family {}
all branch [[143]]
144
family {}
all branch [[144]]
145
family {}
all branch [[145]]
146
family {}
all branch [[146]]
147
family {}
all branch [[147]]
148
family {}
all branch [[148]]
149
family {}
all branch [[149]]
150
family {150: [179, 180]}
all branch [[150, 179], [150, 180]]
151
family {}
all branch [[151]]
152
family {}
all branch [[152]]
153
family {}
all branch [[153]]
154
family {}
all branch [[154]]
155
family {}
all branch [[155]]
156
family {}
all branch [[156]]
157
family {}
all branch [[157]]
158
family {}
all branch [[158]]
159
family {}
all branch [[159]]
160
family {}
all branch [[160]]
161
family {}
all branch [[161]]
162
family {}
all branch [[162]]
163
family {}
all branch [[163]]
164
165
family {}
all branch [[165]]
139
this cell has pca cord
6
166
167
family {}
all branch [[167]]
168
family {}
all branch [[168]]
169
family {}
all branch [[169]]
170
family {}
all branch [[170]]
17

(270, 6)
56
family {}
all branch [[56]]
57
family {}
all branch [[57]]
58
family {}
all branch [[58]]
59
family {}
all branch [[59]]
60
family {}
all branch [[60]]
61
62
63
family {}
all branch [[63]]
64
family {}
all branch [[64]]
65
family {65: [208, 210]}
all branch [[65, 208], [65, 210]]
257
this cell has pca cord
6
(249, 6)
66
family {}
all branch [[66]]
67
family {}
all branch [[67]]
68
family {}
all branch [[68]]
69
family {}
all branch [[69]]
70
family {}
all branch [[70]]
71
family {}
all branch [[71]]
72
family {}
all branch [[72]]
73
family {}
all branch [[73]]
74
family {}
all branch [[74]]
75
family {}
all branch [[75]]
76
family {}
all branch [[76]]
77
family {}
all branch [[77]]
78
family {}
all branch [[78]]
79
family {}
all branch [[79]]
80
family {}
all branch [[80]]
81
family {}
all branch [[81]]
82
family {}
all branch [[82]]
83
family {}
all branch [[83]]
84
family {}
all branch [[84]]
85
family {}
all branch [[85]]
86
family {}
all branch [[86]]
87
family {}
all b

93
family {93: [128, 129]}
all branch [[93, 128], [93, 129]]
94
family {}
all branch [[94]]
95
family {}
all branch [[95]]
96
family {}
all branch [[96]]
97
family {97: [177, 178]}
all branch [[97, 177], [97, 178]]
98
family {}
all branch [[98]]
99
family {}
all branch [[99]]
223
this cell has pca cord
6
(196, 6)
100
family {}
all branch [[100]]
101
family {}
all branch [[101]]
102
family {}
all branch [[102]]
103
family {}
all branch [[103]]
104
family {}
all branch [[104]]
105
family {}
all branch [[105]]
106
family {}
all branch [[106]]
107
108
109
family {}
all branch [[109]]
110
family {}
all branch [[110]]
111
family {}
all branch [[111]]
112
family {}
all branch [[112]]
204
this cell has pca cord
6
113
family {}
all branch [[113]]
114
family {}
all branch [[114]]
115
family {}
all branch [[115]]
116
family {}
all branch [[116]]
117
family {}
all branch [[117]]
118
family {}
all branch [[118]]
119
family {}
all branch [[119]]
120
family {}
all branch [[120]]
121
family {}
all bra

In [11]:
#----------------find long sub traj in a family---------------

mitosis_range=12

feature_list=['mean_intensity','std_intensity','intensity_range','haralick','norm_haralick']
fluor_feature_name=['vimentin_haralick','norm_vimentin_haralick']
for posi in range(1,posi_end+1):
    
    if posi <= 9:
        posi_label = 'XY0' + str(posi)
    else:
        posi_label = 'XY' + str(posi)
    
    print(posi_label)
    
    dir_path=output_path+posi_label+'/'
    cells_path=dir_path+'cells/'
    seg_path=dir_path+'seg/' 
    seg_list=sorted(listdir(seg_path))
 
    df=pd.read_csv(dir_path+'Per_Object_relink.csv')
    am_record=pd.read_csv(dir_path+'am_record.csv')
    traj_df=pd.read_csv(dir_path+'traj_object_num.csv')
    mitosis_df=pd.read_csv(dir_path+'mitosis_record.csv')
    
    with open (cells_path+'fluor_cells', 'rb') as fp:
        cells = pickle.load(fp)

    traj_record=pd.DataFrame(traj_df.loc[:,'1':str(time_span)])
    traj_record=traj_record.values

    traj_quan,traj_len=traj_record.shape[0],traj_record.shape[1]

    for traj_label in range(1,traj_quan+1):
        print(traj_label)
        cur_traj=traj_record[traj_label-1,:]
        traj_start_t=np.where(cur_traj!=-1)[0][0]+1
        
        daughter_flag=find_mother(df,mitosis_df,traj_label)
        if daughter_flag==1:
            continue
        if traj_start_t<=begin_range:
            family_tree={}
            family_tree=find_offspring(df,mitosis_df,family_tree,traj_label=traj_label)
            print('family',family_tree)
            all_branches=list(list(parse(traj_label, family_tree)))
            print('all branch',all_branches)
            
            if len(all_branches)>1:
            
                for branch in all_branches:
                    branch_end_label=branch[-1]
                    branch_end_traj=traj_record[branch_end_label-1,:]
                    branch_end_t=np.where(branch_end_traj!=-1)[0][-1]+1
                    for sub_label in branch:


                        sub_traj=traj_record[sub_label-1,:]
                        sub_start_t=np.where(sub_traj!=-1)[0][0]+1
                        sub_end_t=np.where(sub_traj!=-1)[0][-1]+1
 
                        if (sub_end_t-sub_start_t)>=frag_len_thres and sub_start_t<=begin_range and sub_end_t>=end_range:

                            long_traj=-1*np.ones((time_span,))
                            divide_points=[]
                            traj_name=str(sub_label)+'_'
                            if sub_start_t!=traj_start_t:
                                divide_points.append(sub_start_t)
                                sub_start_ind=sub_start_t+mitosis_range
                            else:
                                sub_start_ind=sub_start_t-1
                                

                            if sub_end_t!=branch_end_t:
                                divide_points.append(sub_end_t)
                                sub_end_ind=sub_end_t-mitosis_range
                            else:
                                sub_end_ind=sub_end_t-1
                                
                            long_traj[max(0,sub_start_ind):min(time_span-1,sub_end_ind)+1]=sub_traj[max(0,sub_start_ind):min(time_span-1,sub_end_ind)+1]
                            
                            whole_branch=long_traj[max(0,sub_start_ind):min(time_span-1,sub_end_ind)+1]
                            if whole_branch.shape[0] == 0:
                                print(sub_start_ind, sub_end_ind)
                            long_traj_loss_ratio=whole_branch[whole_branch==-1].shape[0]*1.0/whole_branch.shape[0]

                            if long_traj_loss_ratio<loss_ratio_thres:

                                traj_feature,traj_contour,traj_cord,\
                                    traj_seri,traj_am_flag,traj_fluor_feature_values,\
                                    traj_fluor_pca_cord=generate_fluor_long_traj(df,cells,am_record,\
                                    traj_df,long_traj,'vimentin',feature_list,fluor_feature_name)
                                traj_sct=fluor_single_cell_traj(traj_seri,traj_contour)
                                traj_sct.set_traj_feature(traj_feature)
                                traj_sct.set_traj_cord(traj_cord)
                                traj_sct.set_traj_divide_points(np.array(divide_points))
                                traj_sct.set_traj_am_flag(traj_am_flag)
                                traj_sct.set_traj_fluor_features('vimentin',feature_list,traj_fluor_feature_values)
                                traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[0],traj_fluor_pca_cord)
                                #traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[1],traj_norm_fluor_pca_cord)
                            if branch_end_t == time_span:
                                with open(sct_path+'sub_sct_'+posi_label+'_'+traj_name+'long_traj', 'wb') as fp:
                                        pickle.dump(traj_sct, fp)
                            else:
                                with open(sct_path+'frag_sub_sct_'+posi_label+'_'+traj_name+'long_traj_label_'+
                                              str(branch_end_label)+'_et_'+str(branch_end_t)+
                                              '_obj_'+str(int(traj_seri[-1][1])), 'wb') as fp:
                                        pickle.dump(traj_sct, fp)

XY01
1
family {}
all branch [[1]]
2
family {2: [34, 35]}
all branch [[2, 34], [2, 35]]
this cell has pca cord
6
(211, 6)
3
family {}
all branch [[3]]
4
family {4: [30, 31]}
all branch [[4, 30], [4, 31]]
5
family {}
all branch [[5]]
6
family {6: [43, 44]}
all branch [[6, 43], [6, 44]]
this cell has pca cord
6
7
family {}
all branch [[7]]
8
family {}
all branch [[8]]
9
family {}
all branch [[9]]
10
family {}
all branch [[10]]
11
family {}
all branch [[11]]
12
family {}
all branch [[12]]
13
family {}
all branch [[13]]
14
family {14: [21, 22], 21: [37, 38], 22: [53, 51], 51: [66, 65]}
all branch [[14, 21, 37], [14, 21, 38], [14, 22, 53], [14, 22, 51, 66], [14, 22, 51, 65]]
15
family {}
all branch [[15]]
16
family {}
all branch [[16]]
17
family {}
all branch [[17]]
18
family {18: [48, 49]}
all branch [[18, 48], [18, 49]]
19
family {}
all branch [[19]]
20
family {}
all branch [[20]]
21
22
23
family {}
all branch [[23]]
24
family {}
all branch [[24]]
25
family {25: [27, 26]}
all branch [[25, 

9
family {}
all branch [[9]]
10
family {10: [67, 66], 67: [195, 198]}
all branch [[10, 67, 195], [10, 67, 198], [10, 66]]
11
family {}
all branch [[11]]
12
family {}
all branch [[12]]
13
family {}
all branch [[13]]
14
family {}
all branch [[14]]
15
family {15: [70, 71], 70: [74, 73]}
all branch [[15, 70, 74], [15, 70, 73], [15, 71]]
16
family {16: [60, 61]}
all branch [[16, 60], [16, 61]]
17
family {17: [80, 81]}
all branch [[17, 80], [17, 81]]
18
family {}
all branch [[18]]
19
family {19: [53, 52], 53: [160, 159]}
all branch [[19, 53, 160], [19, 53, 159], [19, 52]]
20
family {}
all branch [[20]]
21
family {}
all branch [[21]]
22
family {}
all branch [[22]]
23
family {}
all branch [[23]]
24
family {}
all branch [[24]]
25
family {}
all branch [[25]]
26
family {}
all branch [[26]]
27
family {}
all branch [[27]]
28
family {}
all branch [[28]]
29
family {}
all branch [[29]]
30
family {}
all branch [[30]]
31
family {}
all branch [[31]]
32
family {}
all branch [[32]]
33
family {}
all branch 

(335, 6)
10
family {}
all branch [[10]]
11
family {}
all branch [[11]]
12
family {12: [88, 87], 88: [346, 347]}
all branch [[12, 88, 346], [12, 88, 347], [12, 87]]
this cell has pca cord
6
this cell has pca cord
6
this cell has pca cord
6
13
family {13: [40, 39]}
all branch [[13, 40], [13, 39]]
14
family {}
all branch [[14]]
15
family {15: [262, 261]}
all branch [[15, 262], [15, 261]]
16
family {16: [138, 140], 138: [151, 152]}
all branch [[16, 138, 151], [16, 138, 152], [16, 140]]
17
family {}
all branch [[17]]
18
family {}
all branch [[18]]
19
family {19: [149, 148]}
all branch [[19, 149], [19, 148]]
20
family {20: [37, 38]}
all branch [[20, 37], [20, 38]]
21
family {21: [93, 92], 92: [239, 240]}
all branch [[21, 93], [21, 92, 239], [21, 92, 240]]
22
family {}
all branch [[22]]
23
family {}
all branch [[23]]
24
family {}
all branch [[24]]
25
family {}
all branch [[25]]
26
family {}
all branch [[26]]
27
family {}
all branch [[27]]
28
family {}
all branch [[28]]
29
30
31
family {31: [8

(269, 6)
25
family {}
all branch [[25]]
26
family {}
all branch [[26]]
27
28
29
30
31
family {}
all branch [[31]]
32
family {}
all branch [[32]]
33
family {}
all branch [[33]]
34
family {}
all branch [[34]]
35
family {35: [74, 75]}
all branch [[35, 74], [35, 75]]
this cell has pca cord
6
this cell has pca cord
6
36
37
38
family {}
all branch [[38]]
39
family {}
all branch [[39]]
40
family {}
all branch [[40]]
41
family {}
all branch [[41]]
42
family {}
all branch [[42]]
43
family {}
all branch [[43]]
44
family {}
all branch [[44]]
45
family {}
all branch [[45]]
46
family {}
all branch [[46]]
47
family {}
all branch [[47]]
48
family {48: [55, 56]}
all branch [[48, 55], [48, 56]]
49
50
51
52
53
family {}
all branch [[53]]
54
family {}
all branch [[54]]
55
56
57
family {57: [76, 77]}
all branch [[57, 76], [57, 77]]
58
family {}
all branch [[58]]
59
family {}
all branch [[59]]
60
family {}
all branch [[60]]
61
family {}
all branch [[61]]
62
family {}
all branch [[62]]
63
family {}
all bran

1
family {}
all branch [[1]]
2
family {}
all branch [[2]]
3
family {}
all branch [[3]]
4
family {}
all branch [[4]]
5
family {}
all branch [[5]]
6
family {}
all branch [[6]]
7
family {7: [34, 36]}
all branch [[7, 34], [7, 36]]
8
family {8: [45, 46], 45: [136, 137], 136: [212, 215], 137: [231, 228]}
all branch [[8, 45, 136, 212], [8, 45, 136, 215], [8, 45, 137, 231], [8, 45, 137, 228], [8, 46]]
this cell has pca cord
6
9
family {}
all branch [[9]]
10
family {}
all branch [[10]]
11
family {}
all branch [[11]]
12
family {}
all branch [[12]]
13
family {13: [24, 25]}
all branch [[13, 24], [13, 25]]
14
family {14: [30, 29]}
all branch [[14, 30], [14, 29]]
15
family {15: [77, 78]}
all branch [[15, 77], [15, 78]]
16
family {16: [37, 38]}
all branch [[16, 37], [16, 38]]
17
family {}
all branch [[17]]
18
family {18: [74, 75]}
all branch [[18, 74], [18, 75]]
19
family {}
all branch [[19]]
20
family {}
all branch [[20]]
21
family {}
all branch [[21]]
22
family {}
all branch [[22]]
23
family {23: [

...

281
family {}
all branch [[281]]
282
family {}
all branch [[282]]
283
284
285
family {}
all branch [[285]]
286
family {}
all branch [[286]]
287
family {}
all branch [[287]]
288
family {}
all branch [[288]]
289
family {}
all branch [[289]]
290
family {}
all branch [[290]]
291
family {}
all branch [[291]]
292
family {}
all branch [[292]]
293
294
295
family {}
all branch [[295]]
296
family {}
all branch [[296]]
297
family {}
all branch [[297]]
298
family {}
all branch [[298]]
299
family {}
all branch [[299]]
300
family {}
all branch [[300]]
301
family {}
all branch [[301]]
302
family {}
all branch [[302]]
303
family {}
all branch [[303]]
304
family {}
all branch [[304]]
305
family {}
all branch [[305]]
306
family {}
all branch [[306]]
307
family {}
all branch [[307]]
308
family {}
all branch [[308]]
309
family {}
all branch [[309]]
310
family {}
all branch [[310]]
311
family {}
all branch [[311]]
312
family {}
all branch [[312]]
313
family {}
all branch [[313]]
314
315
316
family {}
all b

(55, 6)
this cell has pca cord
6
150
family {}
all branch [[150]]
151
family {}
all branch [[151]]
152
family {}
all branch [[152]]
153
family {}
all branch [[153]]
154
family {}
all branch [[154]]
155
family {}
all branch [[155]]
156
family {}
all branch [[156]]
157
family {}
all branch [[157]]
158
family {}
all branch [[158]]
159
family {}
all branch [[159]]
160
161
162
family {}
all branch [[162]]
163
family {}
all branch [[163]]
164
family {}
all branch [[164]]
165
family {}
all branch [[165]]
166
family {}
all branch [[166]]
167
family {}
all branch [[167]]
168
family {}
all branch [[168]]
169
family {}
all branch [[169]]
170
family {}
all branch [[170]]
171
family {}
all branch [[171]]
172
family {}
all branch [[172]]
173
family {}
all branch [[173]]
174
family {}
all branch [[174]]
175
family {}
all branch [[175]]
176
family {}
all branch [[176]]
177
family {}
all branch [[177]]
178
family {}
all branch [[178]]
179
family {}
all branch [[179]]
180
family {}
all branch [[180]]
18

274
275
276
277
XY17
1
family {}
all branch [[1]]
2
family {}
all branch [[2]]
3
family {}
all branch [[3]]
4
family {}
all branch [[4]]
5
family {}
all branch [[5]]
6
family {}
all branch [[6]]
7
family {}
all branch [[7]]
8
family {}
all branch [[8]]
9
family {}
all branch [[9]]
10
family {}
all branch [[10]]
11
family {}
all branch [[11]]
12
family {}
all branch [[12]]
13
family {}
all branch [[13]]
14
family {}
all branch [[14]]
15
family {15: [59, 60]}
all branch [[15, 59], [15, 60]]
16
family {}
all branch [[16]]
17
family {}
all branch [[17]]
18
family {}
all branch [[18]]
19
family {}
all branch [[19]]
20
family {}
all branch [[20]]
21
family {}
all branch [[21]]
22
family {}
all branch [[22]]
23
family {}
all branch [[23]]
24
family {}
all branch [[24]]
25
family {}
all branch [[25]]
26
family {}
all branch [[26]]
27
family {}
all branch [[27]]
28
family {}
all branch [[28]]
29
family {}
all branch [[29]]
30
family {}
all branch [[30]]
31
family {}
all branch [[31]]
32
family 

family {}
all branch [[184]]
185
family {}
all branch [[185]]
186
family {}
all branch [[186]]
187
family {}
all branch [[187]]
188
family {}
all branch [[188]]
189
family {}
all branch [[189]]
190
family {}
all branch [[190]]
191
family {}
all branch [[191]]
192
family {}
all branch [[192]]
193
family {}
all branch [[193]]
194
family {}
all branch [[194]]
195
family {}
all branch [[195]]
196
family {}
all branch [[196]]
197
family {}
all branch [[197]]
198
family {}
all branch [[198]]
199
family {}
all branch [[199]]
200
family {}
all branch [[200]]
201
family {}
all branch [[201]]
202
family {}
all branch [[202]]
203
204
205
family {}
all branch [[205]]
206
family {}
all branch [[206]]
207
family {}
all branch [[207]]
208
209
family {}
all branch [[209]]
210
211
family {}
all branch [[211]]
212
family {}
all branch [[212]]
213
family {}
all branch [[213]]
214
family {}
all branch [[214]]
215
family {}
all branch [[215]]
216
family {}
all branch [[216]]
217
family {}
all branch [[217]

(153, 6)
15
family {}
all branch [[15]]
16
family {}
all branch [[16]]
17
18
19
family {}
all branch [[19]]
20
family {}
all branch [[20]]
21
22
23
family {}
all branch [[23]]
24
family {24: [61, 62]}
all branch [[24, 61], [24, 62]]
25
family {}
all branch [[25]]
26
family {}
all branch [[26]]
27
28
29
family {}
all branch [[29]]
30
family {}
all branch [[30]]
31
family {}
all branch [[31]]
32
family {}
all branch [[32]]
33
family {}
all branch [[33]]
34
family {34: [117, 116]}
all branch [[34, 117], [34, 116]]
this cell has pca cord
6
this cell has pca cord
6
35
36
37
family {37: [80, 79]}
all branch [[37, 80], [37, 79]]
38
family {}
all branch [[38]]
39
family {}
all branch [[39]]
40
family {40: [82, 81]}
all branch [[40, 82], [40, 81]]
41
family {}
all branch [[41]]
42
family {}
all branch [[42]]
43
family {}
all branch [[43]]
44
family {44: [207, 206]}
all branch [[44, 207], [44, 206]]
45
family {}
all branch [[45]]
46
family {46: [69, 68], 68: [152, 150]}
all branch [[46, 69], [46